# Librairies

In [1]:
import pandas as pd
import numpy as np
import json
from sklearn.preprocessing import MinMaxScaler
from feature_engine.encoding import CountFrequencyEncoder

from keplergl import KeplerGl

import fasttext.util
#fasttext.util.download_model('en', if_exists='ignore')  # English
ft = fasttext.load_model('cc.en.300.bin')
fasttext.util.reduce_model(ft, 1)

# Data

In [2]:
data = pd.read_json('datasets.json')

# Feature Engineering

In [3]:
#We isolate the email provider
data["email"] = data["email"].str.split("@", expand=False).str[-1]

#We split dates, hours, minutes ...
data["user_hour_creation"] = pd.to_datetime(data['user_date_creation']).dt.hour
data["user_min_creation"] = pd.to_datetime(data['user_date_creation']).dt.minute
data["user_quarter_creation"] = pd.to_datetime(data['user_date_creation']).dt.quarter
data["user_day_creation"] = pd.to_datetime(data['user_date_creation']).dt.day
data["user_month_creation"] = pd.to_datetime(data['user_date_creation']).dt.month
data["user_year_creation"] = pd.to_datetime(data['user_date_creation']).dt.year

data["payment_hour"] = pd.to_datetime(data['payment_date']).dt.hour
data["payment_min"] = pd.to_datetime(data['payment_date']).dt.minute
data["payment_quarter"] = pd.to_datetime(data['payment_date']).dt.quarter
data["payment_day"] = pd.to_datetime(data['payment_date']).dt.day
data["payment_month"] = pd.to_datetime(data['payment_date']).dt.month
data["payment_year"] = pd.to_datetime(data['payment_date']).dt.year

#We transform the json from the colum delivery_address to columns for each variable
address = pd.json_normalize(data["delivery_address"])
data = data.join(address)
data = data.drop(['delivery_address'], axis=1)
data.rename(columns = {'name':'neighborhood'}, inplace = True)

#We compare if the billing and delivery address are the same
data['is_billing_delivery_address_equal'] = np.where((data['billing_address'] == data['address']), 1, 0)
data = data.drop(["address","billing_address"], axis=1)

#We compare if the billing and delivery country are the same
data['is_billing_delivery_country_equal'] = np.where((data['state'] == data['billing_country']), 1, 0)

#We compare if the billing and credit card country are the same
data['is_billing_CC_country_equal'] = np.where((data['card_nationality'] == data['billing_country']), 1, 0)

#We compare if the delivery and credit card country are the same
data['is_address_CC_country_equal'] = np.where((data['card_nationality'] == data['state']), 1, 0)

data.head(2)

,user_date_creation,payment_date,adresse_changed_days,browsing_time_seconds,page_visited,number_ticket_opened,items,payment_provider,card_nationality,address_country,...,neighborhood,city,prov,region,latt,longt,is_billing_delivery_address_equal,is_billing_delivery_country_equal,is_billing_CC_country_equal,is_address_CC_country_equal
0,2012-05-07T08:51:40.819Z,2012-05-09T10:47:17.291Z,6,2886,3,6,"[{'name': 'Modern Concrete Cheese', 'quantity'...",Ingenico,RU,NG,...,Nothomb,Attert,Wallonia,Province du Luxembourg,49.77156,5.78773,1,0,0,0
1,2011-09-30T13:36:59.551Z,2011-10-03T16:43:53.522Z,4,1551,10,2,"[{'name': 'Intelligent Steel Bacon', 'quantity...",Authorize.Net,RU,BN,...,Vogelsberg,Wellen,Flanders,Provincie Limburg,50.83333,5.36667,1,0,0,0


In [4]:
#We count the number of commands per user
nbr_command_per_account = data.groupby(['account_id']).count()
nbr_command_per_account.rename(columns = {'adresse_changed_days':'nbr_command_per_account'}, inplace = True)
data = data.merge(nbr_command_per_account["nbr_command_per_account"], left_on="account_id", right_on="account_id")

colum_stat = ["total","page_visited","browsing_time_seconds"]
for column in colum_stat:
    new_mean_column_name = "mean_"+column
    new_median_column_name = "median_"+column
    
    mean = data.groupby(["account_id"]).mean()
    mean.rename(columns = {column: new_mean_column_name}, inplace = True)
    data = data.merge(mean[new_mean_column_name], left_on="account_id", right_on="account_id")
    data[new_mean_column_name] = data[new_mean_column_name].round(0).astype(int)
    
    median = data.groupby(["account_id"]).median()
    median.rename(columns = {column: new_median_column_name}, inplace = True)
    data = data.merge(median[new_median_column_name], left_on="account_id", right_on="account_id")
    data[new_median_column_name] = data[new_median_column_name].round(0).astype(int)

data.head(2)

,user_date_creation,payment_date,adresse_changed_days,browsing_time_seconds,page_visited,number_ticket_opened,items,payment_provider,card_nationality,address_country,...,is_billing_delivery_country_equal,is_billing_CC_country_equal,is_address_CC_country_equal,nbr_command_per_account,mean_total,median_total,mean_page_visited,median_page_visited,mean_browsing_time_seconds,median_browsing_time_seconds
0,2012-05-07T08:51:40.819Z,2012-05-09T10:47:17.291Z,6,2886,3,6,"[{'name': 'Modern Concrete Cheese', 'quantity'...",Ingenico,RU,NG,...,0,0,0,2,538,538,20,20,2585,2585
1,2015-10-27T13:01:13.550Z,2015-11-03T09:52:22.999Z,9,2284,38,3,"[{'name': 'Unbranded Granite Chicken', 'quanti...",Klarna,RU,ES,...,0,0,0,2,538,538,20,20,2585,2585


In [5]:
#We extract the information about the purchased items from the json format in the "items" column 
data_item = pd.json_normalize(data["items"])

for item in range(0,7):
    try:
        item_to_dataframe = pd.json_normalize(data_item[item])
        data = data.merge(item_to_dataframe, left_on="account_id", right_index=True, suffixes=('', str(item)))
    except KeyError:
        print("there is no more than",item+1,"items")
        
columns = data.filter(regex='^quantity',axis=1).columns
for column in columns:
    data[column] = data[column].fillna(0)
    data[column] = data[column].round(0).astype(int)

data.head(2)

there is no more than 5 items
there is no more than 6 items
there is no more than 7 items


,user_date_creation,payment_date,adresse_changed_days,browsing_time_seconds,page_visited,number_ticket_opened,items,payment_provider,card_nationality,address_country,...,mean_browsing_time_seconds,median_browsing_time_seconds,name,quantity,name1,quantity1,name2,quantity2,name3,quantity3
0,2012-05-07T08:51:40.819Z,2012-05-09T10:47:17.291Z,6,2886,3,6,"[{'name': 'Modern Concrete Cheese', 'quantity'...",Ingenico,RU,NG,...,2585,2585,Unbranded Granite Chicken,4,Licensed Metal Shoes,9,Fantastic Concrete Fish,7,NaN,0
1,2015-10-27T13:01:13.550Z,2015-11-03T09:52:22.999Z,9,2284,38,3,"[{'name': 'Unbranded Granite Chicken', 'quanti...",Klarna,RU,ES,...,2585,2585,Unbranded Granite Chicken,4,Licensed Metal Shoes,9,Fantastic Concrete Fish,7,NaN,0


In [6]:
"""
We convert words to vectors thanks to fasttext. We don't use label encoding because new items and thus new words 
can appear after the training corresponding to no category.
"""

columns = data.filter(regex='^name',axis=1).columns
for column in columns:
    embedded_words = []
    for words in data[column].to_list():
        embedded_words.append(ft.get_word_vector(str(words))[0])
    data[column] = embedded_words
    
data.head(2)

,user_date_creation,payment_date,adresse_changed_days,browsing_time_seconds,page_visited,number_ticket_opened,items,payment_provider,card_nationality,address_country,...,mean_browsing_time_seconds,median_browsing_time_seconds,name,quantity,name1,quantity1,name2,quantity2,name3,quantity3
0,2012-05-07T08:51:40.819Z,2012-05-09T10:47:17.291Z,6,2886,3,6,"[{'name': 'Modern Concrete Cheese', 'quantity'...",Ingenico,RU,NG,...,2585,2585,0.023345,4,-0.060146,9,0.018162,7,-0.426326,0
1,2015-10-27T13:01:13.550Z,2015-11-03T09:52:22.999Z,9,2284,38,3,"[{'name': 'Unbranded Granite Chicken', 'quanti...",Klarna,RU,ES,...,2585,2585,0.023345,4,-0.060146,9,0.018162,7,-0.426326,0


In [7]:
"""
We normalize numerical value to avoid larger scales to dominate smaller ones during application of gradient descent, 
and this can lead to many parameters to be undertrained, leading to sub-optimal results.
"""
columns_to_normalize = data.select_dtypes([np.int64]).columns

for column in columns_to_normalize:
    x = data[column].values.reshape(-1, 1) 
    scaler = MinMaxScaler().fit(x)
    data[column] = scaler.transform(x)

data.head(2)

,user_date_creation,payment_date,adresse_changed_days,browsing_time_seconds,page_visited,number_ticket_opened,items,payment_provider,card_nationality,address_country,...,mean_browsing_time_seconds,median_browsing_time_seconds,name,quantity,name1,quantity1,name2,quantity2,name3,quantity3
0,2012-05-07T08:51:40.819Z,2012-05-09T10:47:17.291Z,0.333333,1.000000,0.021739,0.666667,"[{'name': 'Modern Concrete Cheese', 'quantity'...",Ingenico,RU,NG,...,0.964561,0.964561,0.023345,0.428571,-0.060146,1.0,0.018162,1.0,-0.426326,0.0
1,2015-10-27T13:01:13.550Z,2015-11-03T09:52:22.999Z,0.833333,0.785383,0.782609,0.333333,"[{'name': 'Unbranded Granite Chicken', 'quanti...",Klarna,RU,ES,...,0.964561,0.964561,0.023345,0.428571,-0.060146,1.0,0.018162,1.0,-0.426326,0.0


# Data Visualization

In [8]:
#We transform latt & long from string to float to visualize on Kepler their positions
data["latt"] = data["latt"].astype(float)
data["longt"] = data["longt"].astype(float)

#We launch Kepler (https://kepler.gl/)
delivery_map = KeplerGl(height=400)
delivery_map.add_data(data=data[["account_id", "latt", "longt", "total", "payment_date", "city", "delivery_company", "delivery_option", 'delivery_place']], name='deliveries')
delivery_map.save_to_html(file_name='keplergl_map.html')
delivery_map

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter
Map saved to keplergl_map.html!


KeplerGl(data={'deliveries': {'index': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9], 'columns': ['account_id', 'latt', 'long…

# String Encoding

In [9]:
#We drop useless features
data = data.drop(["payment_date","user_date_creation", "items", "neighborhood"], axis=1)

In [10]:
#We manage the string values: payment_provider, card_nationality, address_country, billing_country, email, delivery_company, delivery_place, delivery_option
#First the binary values:
data["voucher"] = data["voucher"].astype(int)
data["subscription"] = data["subscription"].astype(int)

"""
I think a count encoder is better in our situation because we'll pass to the model frequency information. Fraud 
aren't a common behavior, frequencies can help him to identify patterns. 
"""
encoder = CountFrequencyEncoder(encoding_method='frequency', variables=["payment_provider", "card_nationality", "address_country", "billing_country", "email", "delivery_company", "delivery_place", "delivery_option", "state"])
encoder.fit(data)
data = encoder.transform(data)

data.head(2)    

,adresse_changed_days,browsing_time_seconds,page_visited,number_ticket_opened,payment_provider,card_nationality,address_country,billing_country,email,email_changed_days,...,mean_browsing_time_seconds,median_browsing_time_seconds,name,quantity,name1,quantity1,name2,quantity2,name3,quantity3
0,0.333333,1.000000,0.021739,0.666667,0.2,1.0,0.1,0.2,0.4,0.142857,...,0.964561,0.964561,0.023345,0.428571,-0.060146,1.0,0.018162,1.0,-0.426326,0.0
1,0.833333,0.785383,0.782609,0.333333,0.1,1.0,0.1,0.1,0.5,0.000000,...,0.964561,0.964561,0.023345,0.428571,-0.060146,1.0,0.018162,1.0,-0.426326,0.0


In [11]:
"""
We manage the string values remaining: city, prov, region. I choosed to use a vector representation instead of
OneHotEncoder because it will create to many features. Futhermore, it can give information to the model because
city that are close will be spatially close too, for example. 
"""

for column in ["city", "prov", "region"]:
    embedded_words = []
    for words in data[column].to_list():
        embedded_words.append(ft.get_word_vector(str(words))[0])
    data[column] = embedded_words
    
data.head(2)

,adresse_changed_days,browsing_time_seconds,page_visited,number_ticket_opened,payment_provider,card_nationality,address_country,billing_country,email,email_changed_days,...,mean_browsing_time_seconds,median_browsing_time_seconds,name,quantity,name1,quantity1,name2,quantity2,name3,quantity3
0,0.333333,1.000000,0.021739,0.666667,0.2,1.0,0.1,0.2,0.4,0.142857,...,0.964561,0.964561,0.023345,0.428571,-0.060146,1.0,0.018162,1.0,-0.426326,0.0
1,0.833333,0.785383,0.782609,0.333333,0.1,1.0,0.1,0.1,0.5,0.000000,...,0.964561,0.964561,0.023345,0.428571,-0.060146,1.0,0.018162,1.0,-0.426326,0.0


In [12]:
data.to_csv('extract.csv')